In [2]:
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install lapx

Defaulting to user installation because normal site-packages is not writeable


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.5 MB 932.9 kB/s eta 0:00:02
   --------------------- ------------------ 0.8/1.5 MB 882.6 kB/s eta 0:00:01
   --------------------- ------------------ 0.8/1.5 MB 882.6 kB/s eta 0:00:01
   ---------------------------- ----------- 1.0/1.5 MB 898.8 kB/s eta 0:00:01
   ----------------------------------- ---- 1.3/1.5 MB 932.1 kB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 903.8 kB/s eta 0:00:00


In [1]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
  Using cached ultralytics-8.3.111-py3-none-any.whl.metadata (37 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
Using cached ultralytics-8.3.111-py3-none-any.whl (978 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
Using cached torch-2.6.0-cp312-cp312-win_amd64.whl (204.1 MB)
Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl (1.6 MB)
Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [6]:
import numpy as np
from numpy import linalg as LA
import cv2
import os
import glob
from random import randrange
from tqdm import tqdm
from SFSORT import SFSORT
from ultralytics import YOLO

class boundaryLine:
    def __init__(self, line_id, line=[(0, 0), (0, 0)]):
        self.id = line_id
        self.p0 = line[0]
        self.p1 = line[1]
        self.color = (0, 255, 255)
        self.lineThinkness = 4
        self.textColor = (0, 255, 255)
        self.textSize = 4
        self.textThinkness = 2
        self.count1 = 0
        self.count2 = 0

def line_vectorize(point1, point2):
    a = point2[0] - point1[0]
    b = point2[1] - point1[1]
    return [a, b]

def calcVectorAngle(point1, point2, point3, point4):
    u = np.array(line_vectorize(point1, point2))
    v = np.array(line_vectorize(point3, point4))
    i = np.inner(u, v)
    n = LA.norm(u) * LA.norm(v)
    c = i / n
    a = np.rad2deg(np.arccos(np.clip(c, -1.0, 1.0)))
    return a if u[0]*v[1]-u[1]*v[0] < 0 else 360 - a

def checkIntersect(p1, p2, p3, p4):
    tc1 = (p1[0] - p2[0]) * (p3[1] - p1[1]) + (p1[1] - p2[1]) * (p1[0] - p3[0])
    tc2 = (p1[0] - p2[0]) * (p4[1] - p1[1]) + (p1[1] - p2[1]) * (p1[0] - p4[0])
    td1 = (p3[0] - p4[0]) * (p1[1] - p3[1]) + (p3[1] - p4[1]) * (p3[0] - p1[0])
    td2 = (p3[0] - p4[0]) * (p2[1] - p3[1]) + (p3[1] - p4[1]) * (p3[0] - p2[0])
    return tc1 * tc2 < 0 and td1 * td2 < 0

def checkLineCross(boundary_line, trajectory, track_id):
    traj_p0 = (trajectory[0], trajectory[1])
    traj_p1 = (trajectory[2], trajectory[3])
    bLine_p0 = (boundary_line.p0[0], boundary_line.p0[1])
    bLine_p1 = (boundary_line.p1[0], boundary_line.p1[1])
    intersect = checkIntersect(traj_p0, traj_p1, bLine_p0, bLine_p1)
    if intersect:
        angle = calcVectorAngle(traj_p0, traj_p1, bLine_p0, bLine_p1)
        if 15 < angle < 165:
            boundary_line.count1 += 1
        elif 195 < angle < 345:
            boundary_line.count2 += 1
        return f'ID {track_id} crossed line {boundary_line.id}!'
    return None

def drawBoundaryLine(img, line):
    x1, y1 = line.p0
    x2, y2 = line.p1
    cv2.line(img, (x1, y1), (x2, y2), line.color, line.lineThinkness)
    cv2.putText(img, str(line.count1), (x1, y1), cv2.FONT_HERSHEY_PLAIN, line.textSize, line.textColor, line.textThinkness)
    cv2.putText(img, str(line.count2), (x2, y2), cv2.FONT_HERSHEY_PLAIN, line.textSize, line.textColor, line.textThinkness)
    cv2.drawMarker(img, (x1, y1), line.color, cv2.MARKER_TRIANGLE_UP, 16, 4)
    cv2.drawMarker(img, (x2, y2), line.color, cv2.MARKER_TILTED_CROSS, 16, 4)

def drawBoundaryLines(img, boundaryLines):
    for line in boundaryLines:
        drawBoundaryLine(img, line)

folder_path = r'C:\\Users\\user\\Desktop\\intrution'
video_files = glob.glob(os.path.join(folder_path, "*.avi"))
if len(video_files) != 1:
    print(video_files)
    raise ValueError("There should be exactly one .mp4 file in the folder.")
video_path = video_files[0]

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
if not ret:
    raise RuntimeError("Failed to read first frame.")

# Interactive drawing
boundary_points = []
drawing = False
current_line = []

def draw_line(event, x, y, flags, param):
    global drawing, current_line, boundary_points
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        current_line = [(x, y)]
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        temp_img = frame.copy()
        cv2.line(temp_img, current_line[0], (x, y), (0, 255, 255), 2)
        cv2.imshow("Draw Lines (up to 3)", temp_img)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        current_line.append((x, y))
        boundary_points.append(tuple(current_line))
        cv2.line(frame, current_line[0], current_line[1], (0, 255, 255), 2)
        cv2.imshow("Draw Lines (up to 3)", frame)

cv2.namedWindow("Draw Lines (up to 3)")
cv2.setMouseCallback("Draw Lines (up to 3)", draw_line)

print("Draw up to 3 boundary lines. Press 'q' to confirm.")
while True:
    cv2.imshow("Draw Lines (up to 3)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q') or len(boundary_points) >= 3:
        break
cv2.destroyWindow("Draw Lines (up to 3)")

bLines = [boundaryLine(i+1, list(pts)) for i, pts in enumerate(boundary_points)]
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # rewind

# The rest of your tracking and processing pipeline follows from here...
# (Insert your tracking loop here, reusing `bLines`, `cap`, etc.)

model_path = os.path.join(folder_path, 'best.pt')
model = YOLO(model_path, 'detect')


output_video_path = os.path.join(folder_path, "tracked_output_MOT.mp4")



# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Tracker parameters (adjust as needed)
tracker_arguments = {
    "dynamic_tuning": True,
    "high_th": 0.5,
    "low_th": 0.2,
    "match_th_first": 0.4,
    "match_th_first_m": 0.05,
    "match_th_second": 0.4,
    "marginal_timeout": int(1.1*frame_rate),  # Example value; adjust based on frame rate
    "central_timeout": int(1.4*frame_rate),  # Example value; adjust based on frame rate
    "horizontal_margin": int(0.1*frame_width),  # Example value
    "vertical_margin": int(0.1*frame_height),  # Example value
    "frame_width": frame_width,  # To be set dynamically
    "frame_height": frame_height  # To be set dynamically
}

tracker = SFSORT(tracker_arguments)


# Setup video writer for output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (frame_width, frame_height))
colors = {}
intruders = {}
tracks = {}
for frame_no in tqdm(range(frame_count)):
    ret, frame = cap.read()
    if not ret:
        print(f"End of video reached or error reading frame {frame_no}.")
        break
    prediction = model.predict(frame, imgsz=(800,1440),
                               conf=0.1, iou=0.45, half=False,
                               max_det=60, classes=0, verbose=False)
    
    # exclude extra info from the predictions
    predictionResults = prediction[0].boxes.cpu().numpy()





    box_list = predictionResults.xyxy
    score_list = predictionResults.conf

    if box_list.shape[0] == 0:
      print(f'no detection found in {frame_no+1}')
      drawBoundaryLines(frame,bLines)
      # cv2.imwrite(os.path.join(folder_path,str(frame_no+1) +'.jpg'),frame)
      out.write(frame)
      continue

    # Update tracker with detections
    track_list = tracker.update(box_list, score_list)


    # Annotate frame with tracking results
    for track in track_list:
        bbox, track_id = track[0], int(track[1])
        if track_id not in tracks:
          tracks[track_id] = bbox
        prev_box = tracks[track_id]
        
        
        x0, y0, x1, y1 = map(int, bbox)
        cx,cy = (x0+x1)/2 , (y0+y1)/2

        
        prev_x0,prev_y0,prev_x1,prev_y1 = map(int, prev_box)
        prev_cx,prev_cy = (prev_x0+prev_x1)/2 , (prev_y0+prev_y1)/2
        
        crossed = [None]*len(bLines)
        for j,line in enumerate(bLines):
          crossed[j] = checkLineCross(line, [prev_cx,prev_cy, cx,cy], track_id)

        if track_id not in colors:
          colors[track_id] =  (randrange(255), randrange(255), randrange(255))

        color = colors[track_id]
        track_thickness = 1
        if any(crossed):
          if track_id not in intruders:
            intruders[track_id] = [[cx,cy]]
          message = [m for m in crossed if m is not None]
          cv2.putText(frame, str(message[0]), (20,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
          print(frame_no+1)
          print(str(message[0]))

        # drawBoundaryLines(frame,bLines)
        
        if track_id in intruders:
          color = (0,0,255)
          track_thickness = 2
          intruders[track_id].append([cx,cy])
          cv2.polylines(frame, np.array([intruders[track_id]], np.int32), False, (255,255,255), 4)

        frame = cv2.rectangle(frame, (x0, y0), (x1, y1), color, track_thickness)
        cv2.putText(frame, str(track_id), (x0, y0-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, track_thickness)
        tracks[track_id] = bbox

    drawBoundaryLines(frame,bLines)
    # cv2.imwrite(os.path.join(folder_path,str(frame_no+1) +'.jpg'),frame)
    out.write(frame)

# Release resources for this video
cap.release()
out.release()
print(f"Tracked video saved at: {output_video_path}")


Draw up to 3 boundary lines. Press 'q' to confirm.


  7%|▋         | 77/1050 [00:35<07:07,  2.27it/s]

77
ID 23 crossed line 1!


  9%|▉         | 99/1050 [00:45<07:11,  2.20it/s]

99
ID 20 crossed line 2!


...

562
ID 10 crossed line 2!


 98%|█████████▊| 1025/1050 [08:20<00:11,  2.16it/s]

1025
ID 80 crossed line 1!


100%|██████████| 1050/1050 [08:32<00:00,  2.05it/s]

Tracked video saved at: C:\\Users\\user\\Desktop\\intrution\tracked_output_MOT.mp4
